In [1]:
using Plots
using CSV
using ControlSystems

gr()

Plots.GRBackend()

In [2]:
#体を揺すって脚を出す歩容
#確定版2019/01/20

zc=0.11 #0.12
g=9.8
dt=0.0025
L=0.15 #0.16
l=0.025 #歩幅
#x=0.03
ft=0.05 #最初の全脚支持期間

#support time
Tsup=Int32(0.4/dt)
#both foot support point time
Tdl=Int32(0.0025/dt) 
#preview step
M=800 #600
            #r             c         l
real_sx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l; #第一ZMP
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l; #一度動かない瞬間を作る
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;#第二zmp
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;
         
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;#stop time
    
        (1/2)*sqrt(3)*L+l; 0.0+l; (1/2)*sqrt(3)*L+l;#第3zmp
    
        (1/2)*sqrt(3)*L+l; 0.0+l; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L+l; 0.0+l; (1/2)*sqrt(3)*L+l;
        
        ] #  rcl

real_sy=[0.0; L/2; L;
    
        0.0; L/2; L;
         0.0;  L/2; L;
    
        0.0;  L/2; L; #動かない瞬間を作る
        
        0.0;  L/2; L;
        0.0;  L/2; L;
        0.0;  L/2; L;
        
        0.0;  L/2; L; #sptime
        0.0;  L/2; L;
        
        0.0;  L/2; L;#動かない期間
    
        0.0;  L/2; L;
        0.0;  L/2; L;
        0.0;  L/2; L;

        0.0;  L/2; L;
         ]

pltsx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L] #plot用
pltsy=[0.0; L/2; L]

leg_sign=[0;3;0;0;0;0;2;0;0;0;0;1;0;0;]#上げる脚とステップの関係 #left:3 right:1 center:

#初期姿勢
f_zmp_x=L/sqrt(3)   #abs(real_sx[1]-real_sx[2])/2
f_zmp_y=L/2+0.00

e_zmp_x=L/sqrt(3)+l
e_zmp_y=L/2+0.00 #追加

#Virtual support point
sx=[f_zmp_x
    real_sx[8]+abs(real_sx[4]-real_sx[8])/2+0.005+0.004
    L/sqrt(3)+(l/3)
    
    L/sqrt(3)+(l/3) #動かない瞬間をつくる
    
    L/sqrt(3)+(l/3)+0.022 #sp time
    L/sqrt(3)+(l/3)+0.022 #stop time
    real_sx[1]+(l/2)+0.013 #支持脚線の第二ZMP
    L/sqrt(3)+(l/3)*2#+0.06
    
    L/sqrt(3)+(l/3)*2#+0.01 #stop time
    L/sqrt(3)+(l/3)*2 #sp time
    L/sqrt(3)+(l/3)*2 #stop time
    
    real_sx[1]/2+l+0.00 #第三zmp 0.005
    L/sqrt(3)+l+0.005 
    
    e_zmp_x]

sy=[f_zmp_y
    abs(real_sy[7]-real_sy[8])/2+0.005+0.003
    L/2
    
    L/2 #一度何もしない瞬間を作る
    
    L/2 #sp time
    L/2#stoptime
    L/2 #ZMP2
    L/2
    
    L/2#stop time
    L/2#sptime
    L/2#stoptime
    
    real_sy[2]+abs(real_sy[2]-real_sy[3])/2-0.004 #第三zmp
    L/2
    
    e_zmp_y]

vpoint_sum=length(sx)
full_leg_step_sum=vpoint_sum-1

#to generate trajectory of ZMP when both foot support point 
x_tdl_a=zeros(Float64,full_leg_step_sum)
x_tdl_b=zeros(Float64,full_leg_step_sum)
y_tdl_a=zeros(Float64,full_leg_step_sum)
y_tdl_b=zeros(Float64,full_leg_step_sum)

for i in 1:full_leg_step_sum
    x_tdl_a[i]=(sx[i]-sx[i+1])/(-Tdl*dt)
    x_tdl_b[i]=sx[i]-x_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

for i in 1:full_leg_step_sum
    y_tdl_a[i]=(sy[i]-sy[i+1])/(-Tdl*dt)
    y_tdl_b[i]=sy[i]-y_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

#target ZMP vector
px_ref=zeros(vpoint_sum*Tsup+M+1 + full_leg_step_sum*Tdl)
py_ref=zeros(px_ref)

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            px_ref[k+step]=sx[i]
        end
    else
        for k in 1:Tsup
            px_ref[k+step]=sx[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            px_ref[s+step]=x_tdl_a[i]*(s+step)*dt+x_tdl_b[i]
        end
        step=step+Tdl
    end
end

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            py_ref[k+step]=sy[i]
        end
    else
        for k in 1:Tsup
            py_ref[k+step]=sy[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            py_ref[s+step]=y_tdl_a[i]*(s+step)*dt+y_tdl_b[i]
        end
        step=step+Tdl
    end
end

#total step
k_max=length(px_ref)-M
#computed ZMP vector
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max);

In [3]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[25000 0 0 0
    0 1 0 0
    0 0 1 0
    0 0 0 1]
R=1.0

#state vector
x=[ sx[1]
    0.0
    0.0]

y=[ sy[1]
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0];

In [4]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #to record for plot
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #to record for plot
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 

xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y

dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR
old_x=x
old_y=y
xu=0
yu=0
P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]^(-1)*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)[1]^(-1)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)[1]^(-1)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 0.0

In [5]:
for i in 1:k_max-1
    
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]    
    k+=1
    x_plot_log[k+1]=x[1] #to record for plot
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #to record for plot
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    dx=x-old_x
    dy=y-old_y
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    old_x=x
    old_y=y
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end

In [6]:
plot(t[1:2200],px_ref[1:2200],lab="Target ZMP",linewidth=3,linecolor="black")
plot!(t[1:2200],x_plot_log[1:2200],lab="CoM",linewidth=3,linecolor="blue")
plot!(t[1:2200],px[1:2200],lab="ZMP",linewidth=3,linecolor="red")
a=[0.18 0.18]
b=[10 10]
plot!(b,a)
#plot!(xlab="time[ms]",yl\ab="ZMP_x[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 1000 
 
 
 1250 
 
 
 1500 
 
 
 1750 
 
 
 2000 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 0.150 
 
 
 0.175 
 
<polyline clip-path="url(#clip0102)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,322.512 39.6214,322.512 39.8728,322.512 40.1242,322.512 40.3755,322.512 40.6269,322.512 40.8783,322.512 41.1296,322.512 41.381,322.512 41.6324,322.512 
 41.8837,322.512 42.1351,322.512 42.3865,322.512 42.6378,322.512 42.8892,322.512 43.1406,322.512 43.3919,322.512 43.6433,322.512 43.8947,322.512 44.1461,322.512 
 44.3974,322.512 44.6488,322.512 44.9002,322.512 45.1515,322.512 45.4029,322.512 45.6543,322.512 45.9056,322.512 46.157,322.512 46.4084,322.512 46.6597,322.512 
 46.9111,322.512 47.1625,322.512 47.4138,322.512 47.6652,322.512 47.9166,322.512 48.1679,322.512 48.4193,322.512 48.6707,322.512 48.922,322.512 49.1734,322.512 
 49.4248,322.512 49.6761,322.512 49.9275,322.512 50.1789,322.512 50.4302,322.512 50.6816,322.512 50.933,322.512 51.1843,322.512 51.4357,322.512 51.6871,322.512 
 51.9384,322.512 52.1898,322.512 52.4412,322.512 52.6925,322.512 52.9439,322.512 53.1953,322.512 53.4466,322.512 53.698,322.512 53.9494,322.512 54.2007,322.512 
 54.4521,322.512 54.7035,322.512 54.9548,322.512 55.2062,322.512 55.4576,322.512 55.7089,322.512 55.9603,322.512 56.2117,322.512 56.463,322.512 56.7144,322.512 
 56.9658,322.512 57.2171,322.512 57.4685,322.512 57.7199,322.512 57.9712,322.512 58.2226,322.512 58.474,322.512 58.7253,322.512 58.9767,322.512 59.2281,322.512 
 59.4794,322.512 59.7308,322.512 59.9822,322.512 60.2335,322.512 60.4849,322.512 60.7363,322.512 60.9876,322.512 61.239,322.512 61.4904,322.512 61.7417,322.512 
 61.9931,322.512 62.2445,322.512 62.4958,322.512 62.7472,322.512 62.9986,322.512 63.2499,322.512 63.5013,322.512 63.7527,322.512 64.004,322.512 64.2554,322.512 
 64.5068,322.512 64.7581,322.512 65.0095,322.512 65.2609,322.512 65.5122,322.512 65.7636,322.512 66.015,322.512 66.2663,322.512 66.5177,322.512 66.7691,322.512 
 67.0204,322.512 67.2718,322.512 67.5232,322.512 67.7745,322.512 68.0259,322.512 68.2773,322.512 68.5286,322.512 68.78,322.512 69.0314,322.512 69.2827,322.512 
 69.5341,322.512 69.7855,322.512 70.0368,322.512 70.2882,322.512 70.5396,322.512 70.7909,322.512 71.0423,322.512 71.2937,322.512 71.545,322.512 71.7964,322.512 
 72.0478,322.512 72.2991,322.512 72.5505,322.512 72.8019,322.512 73.0532,322.512 73.3046,322.512 73.556,322.512 73.8073,322.512 74.0587,322.512 74.3101,322.512 
 74.5615,322.512 74.8128,322.512 75.0642,322.512 75.3156,322.512 75.5669,322.512 75.8183,322.512 76.0697,322.512 76.321,322.512 76.5724,322.512 76.8238,322.512 
 77.0751,322.512 77.3265,322.512 77.5779,322.512 77.8292,322.512 78.0806,322.512 78.332,322.512 78.5833,322.512 78.8347,322.512 79.0861,322.512 79.3374,322.512 
 79.5888,365.13 79.8402,365.13 80.0915,365.13 80.3429,365.13 80.5943,365.13 80.8456,365.13 81.097,365.13 81.3484,365.13 81.5997,365.13 81.8511,365.13 
 82.1025,365.13 82.3538,365.13 82.6052,365.13 82.8566,365.13 83.1079,365.13 83.3593,365.13 83.6107,365.13 83.862,365.13 84.1134,365.13 84.3648,365.13 
 84.6161,365.13 84.8675,365.13 85.1189,365.13 85.3702,365.13 85.6216,365.13 85.873,365.13 86.1243,365.13 86.3757,365.13 86.6271,365.13 86.8784,365.13 
 87.1298,365.13 87.3812,365.13 87.6325,365.13 87.8839,365.13 88.1353,365.13 88.3866,365.13 88.638,365.13 88.8894,365.13 89.1407,365.13 89.3921,365.13 
 89.6435,365.13 89.8948,365.13 90.1462,365.13 90.3976,365.13 90.6489,365.13 90.9003,365.13 91.1517,365.13 91.403,365.13 91.6544,365.13 91.9058,365.13 
 92.1571,365.13 92.4085,365.13 92.6599,365.13 92.9112,365.13 93.1626,365.13 93.414,365.13 93.6653,365.13 93.9167,365.13 94.1681,365.13 94.4194,365.13 
 94.6708,365.13 94.9222,365.13 95.1735,365.13 95.4249,365.13 95.6763,365.13 95.9276,365.13 96.179,365.13 96.4304,365.13 96.6817,

In [7]:
plot(t,py_ref,lab="Target ZMP y",linewidth=3,linecolor="black")
plot!(t,y_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t[1:2200],py[1:2200],lab="ZMP y",linewidth=3,linecolor="red")
a=[0.165 0.165]
b=[10 10]
plot!(b,a)
#plot!(xlab="time[ms]",ylab="ZMP[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 300 
 
 
 600 
 
 
 900 
 
 
 1200 
 
 
 1500 
 
 
 1800 
 
 
 2100 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 0.150 
 
<polyline clip-path="url(#clip0202)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,204.581 39.6154,204.581 39.8608,204.581 40.1061,204.581 40.3514,204.581 40.5968,204.581 40.8421,204.581 41.0875,204.581 41.3328,204.581 41.5782,204.581 
 41.8235,204.581 42.0688,204.581 42.3142,204.581 42.5595,204.581 42.8049,204.581 43.0502,204.581 43.2956,204.581 43.5409,204.581 43.7862,204.581 44.0316,204.581 
 44.2769,204.581 44.5223,204.581 44.7676,204.581 45.0129,204.581 45.2583,204.581 45.5036,204.581 45.749,204.581 45.9943,204.581 46.2397,204.581 46.485,204.581 
 46.7303,204.581 46.9757,204.581 47.221,204.581 47.4664,204.581 47.7117,204.581 47.9571,204.581 48.2024,204.581 48.4477,204.581 48.6931,204.581 48.9384,204.581 
 49.1838,204.581 49.4291,204.581 49.6744,204.581 49.9198,204.581 50.1651,204.581 50.4105,204.581 50.6558,204.581 50.9012,204.581 51.1465,204.581 51.3918,204.581 
 51.6372,204.581 51.8825,204.581 52.1279,204.581 52.3732,204.581 52.6186,204.581 52.8639,204.581 53.1092,204.581 53.3546,204.581 53.5999,204.581 53.8453,204.581 
 54.0906,204.581 54.336,204.581 54.5813,204.581 54.8266,204.581 55.072,204.581 55.3173,204.581 55.5627,204.581 55.808,204.581 56.0533,204.581 56.2987,204.581 
 56.544,204.581 56.7894,204.581 57.0347,204.581 57.2801,204.581 57.5254,204.581 57.7707,204.581 58.0161,204.581 58.2614,204.581 58.5068,204.581 58.7521,204.581 
 58.9975,204.581 59.2428,204.581 59.4881,204.581 59.7335,204.581 59.9788,204.581 60.2242,204.581 60.4695,204.581 60.7148,204.581 60.9602,204.581 61.2055,204.581 
 61.4509,204.581 61.6962,204.581 61.9416,204.581 62.1869,204.581 62.4322,204.581 62.6776,204.581 62.9229,204.581 63.1683,204.581 63.4136,204.581 63.659,204.581 
 63.9043,204.581 64.1496,204.581 64.395,204.581 64.6403,204.581 64.8857,204.581 65.131,204.581 65.3763,204.581 65.6217,204.581 65.867,204.581 66.1124,204.581 
 66.3577,204.581 66.6031,204.581 66.8484,204.581 67.0937,204.581 67.3391,204.581 67.5844,204.581 67.8298,204.581 68.0751,204.581 68.3205,204.581 68.5658,204.581 
 68.8111,204.581 69.0565,204.581 69.3018,204.581 69.5472,204.581 69.7925,204.581 70.0378,204.581 70.2832,204.581 70.5285,204.581 70.7739,204.581 71.0192,204.581 
 71.2646,204.581 71.5099,204.581 71.7552,204.581 72.0006,204.581 72.2459,204.581 72.4913,204.581 72.7366,204.581 72.982,204.581 73.2273,204.581 73.4726,204.581 
 73.718,204.581 73.9633,204.581 74.2087,204.581 74.454,204.581 74.6994,204.581 74.9447,204.581 75.19,204.581 75.4354,204.581 75.6807,204.581 75.9261,204.581 
 76.1714,204.581 76.4167,204.581 76.6621,204.581 76.9074,204.581 77.1528,204.581 77.3981,204.581 77.6435,204.581 77.8888,204.581 78.1341,204.581 78.3795,204.581 
 78.6248,269.058 78.8702,269.058 79.1155,269.058 79.3609,269.058 79.6062,269.058 79.8515,269.058 80.0969,269.058 80.3422,269.058 80.5876,269.058 80.8329,269.058 
 81.0782,269.058 81.3236,269.058 81.5689,269.058 81.8143,269.058 82.0596,269.058 82.305,269.058 82.5503,269.058 82.7956,269.058 83.041,269.058 83.2863,269.058 
 83.5317,269.058 83.777,269.058 84.0224,269.058 84.2677,269.058 84.513,269.058 84.7584,269.058 85.0037,269.058 85.2491,269.058 85.4944,269.058 85.7397,269.058 
 85.9851,269.058 86.2304,269.058 86.4758,269.058 86.7211,269.058 86.9665,269.058 87.2118,269.058 87.4571,269.058 87.7025,269.058 87.9478,269.058 88.1932,269.058 
 88.4385,269.058 88.6839,269.058 88.9292,269.058 89.1745,269.058 89.4199,269.058 89.6652,269.058 89.9106,269.058 90.1559,269.058 90.4012,269.058 90.6466,269.058 
 90.8919,269.058 91.1373,269.058 91.3826,269.058 91.628,269.058 91.8733,269.058 92.1186,269.058 92.364,269.058 92.6093,269.058 92.8547,269.058 93.1,269.058 
 93.3454,269.058 93.5907,269.058 93.836,269.058 94.081

In [8]:
plot(x_plot_log,y_plot_log,lab="CoM",linewidth=1,linecolor="blue")
plot!(px,py,lab="ZoM",linewidth=1,linecolor="black") #追加
plot!(real_sx,real_sy,st=:scatter)
plot!(sx,sy,st=:scatter)
a=[0.15 0.15]
b=[0.19 0.19]
plot!(b,a)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 0.150 
 
 
 0.175 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 0.12 
 
 
 0.14 
 
<polyline clip-path="url(#clip0302)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 292.701,188.189 292.701,188.189 292.701,188.189 292.701,188.189 292.701,188.189 292.701,188.189 292.7,188.189 292.7,188.19 292.7,188.19 292.699,188.191 
 292.699,188.192 292.698,188.194 292.697,188.196 292.696,188.198 292.694,188.201 292.692,188.205 292.69,188.209 292.688,188.214 292.685,188.22 292.681,188.227 
 292.677,188.234 292.673,188.243 292.668,188.252 292.663,188.263 292.657,188.275 292.65,188.288 292.643,188.302 292.635,188.317 292.627,188.334 292.617,188.352 
 292.608,188.371 292.597,188.392 292.586,188.414 292.574,188.437 292.561,188.462 292.547,188.489 292.533,188.517 292.518,188.546 292.502,188.578 292.485,188.61 
 292.468,188.645 292.449,188.681 292.43,188.718 292.41,188.757 292.389,188.798 292.367,188.841 292.345,188.885 292.321,188.931 292.297,188.979 292.272,189.028 
 292.246,189.079 292.219,189.132 292.191,189.187 292.162,189.243 292.132,189.301 292.101,189.362 292.07,189.423 292.037,189.487 292.004,189.553 291.969,189.62 
 291.934,189.69 291.897,189.761 291.86,189.834 291.821,189.91 291.782,189.987 291.741,190.066 291.7,190.147 291.657,190.231 291.613,190.317 291.568,190.404 
 291.522,190.494 291.475,190.587 291.427,190.681 291.377,190.778 291.327,190.878 291.275,190.979 291.221,191.084 291.167,191.191 291.111,191.3 291.053,191.412 
 290.995,191.527 290.934,191.645 290.872,191.766 290.809,191.89 290.744,192.017 290.678,192.147 290.609,192.281 290.539,192.418 290.468,192.558 290.394,192.702 
 290.319,192.85 290.241,193.001 290.162,193.157 290.08,193.316 289.997,193.48 289.911,193.647 289.823,193.82 289.732,193.997 289.639,194.178 289.544,194.365 
 289.446,194.556 289.345,194.753 289.242,194.954 289.136,195.162 289.027,195.374 288.915,195.593 288.801,195.817 288.683,196.048 288.561,196.284 288.437,196.527 
 288.309,196.777 288.178,197.033 288.044,197.296 287.905,197.566 287.764,197.843 287.618,198.128 287.469,198.419 287.315,198.719 287.158,199.026 286.997,199.341 
 286.831,199.664 286.662,199.995 286.488,200.334 286.31,200.682 286.128,201.038 285.941,201.402 285.75,201.775 285.554,202.157 285.354,202.548 285.15,202.947 
 284.941,203.355 284.727,203.772 284.509,204.198 284.287,204.632 284.06,205.075 283.828,205.527 283.592,205.988 283.352,206.457 283.108,206.935 282.859,207.421 
 282.606,207.915 282.349,208.418 282.088,208.928 281.823,209.446 281.554,209.971 281.282,210.503 281.006,211.042 280.727,211.588 280.445,212.139 280.16,212.697 
 279.872,213.26 279.582,213.828 279.289,214.401 278.995,214.978 278.698,215.559 278.4,216.143 278.101,216.73 277.8,217.319 277.499,217.91 277.197,218.502 
 276.895,219.095 276.593,219.688 276.291,220.28 275.99,220.872 275.69,221.464 275.39,222.053 275.091,222.641 274.794,223.227 274.498,223.81 274.204,224.389 
 273.912,224.965 273.623,225.537 273.336,226.105 273.051,226.668 272.77,227.225 272.491,227.777 272.216,228.323 271.945,228.863 271.676,229.396 271.412,229.922 
 271.152,230.44 270.895,230.952 270.643,231.456 270.396,231.951 270.152,232.439 269.914,232.918 269.68,233.389 269.45,233.851 269.226,234.304 269.006,234.749 
 268.791,235.184 268.582,235.61 268.377,236.027 268.177,236.435 267.983,236.833 267.793,237.222 267.609,237.602 267.43,237.972 267.256,238.333 267.087,238.684 
 266.924,239.027 266.765,239.359 266.612,239.683 266.464,239.997 266.321,240.302 266.183,240.598 266.05,240.885 265.922,241.163 265.799,241.432 265.681,241.693 
 265.568,241.944 265.46,242.187 265.356,242.421 265.258,242.647 265.164,242.864 265.075,243.073 264.99,243.274 264.911,243.467 264.836,243.652 264.765,243.829 
 264.699,243.998 264.638,244.159 264.581,244.313 

In [9]:
function trj_generator(sx,xv,tf,te,dt,legnum)#xの遊脚軌道
    #
    #条件ベクトル
    xf=sx[legnum,tf]
    xe=sx[legnum,te]
    vf=xv[tf]*-1
    ve=xv[te]*-1
    A=[xf; xe; vf; ve]
    
    tf=round(tf*dt,6) #時間に戻す
    te=round(te*dt,6) #第6位で切り捨て
        
    B=[tf^3 tf^2 tf 1
        te^3 te^2 te 1
        3tf^2 2tf 1 0
        3te^2 2te 1 0]

    C=inv(B)*A
    

    #dt=0.01 #離散時間
    t=[i for i in tf:dt:te] #描画時間
    x=C[1]*.^(t,3)+C[2]*.^(t,2)+C[3]*t+C[4] #位置
    v=3*C[1]*.^(t,2)+2*C[2]*t+C[3] #速度
    a=6*C[1]*t+2*C[2]
    
    return x
    
end

function trj_generator_z(zh,zf,ze,ve,tf,te,dt)
    #support time 0.5
    tf=round(tf*dt,7) #時間に戻す
    te=round(te*dt,7) #第6位で切り捨て
    th=round((te-tf)/2+tf,7)
    
    zh=zf+zh
   
    A_z=[zh; ze; zf; ve]

    B_z=[th^3 th^2 th 1
        te^3 te^2 te 1
        tf^3 tf^2 tf 1
        3te^2 2te 1 0]
    C_z=inv(B_z)*A_z
    
    t=[i for i in tf:dt:te]; #描画時間
     z=C_z[1]*.^(t,3)+C_z[2]*.^(t,2)+C_z[3]*t+C_z[4] #位置
    v=3*C_z[1]*.^(t,2)+2*C_z[2]*t+C_z[3] #速度
    a=6*C_z[1]*t+2*C_z[2]
    return z
end



In [10]:
#脚先位置を重心の座標系に変換する
com_trjx=copy(x_plot_log) #重心位置
com_trjy=copy(y_plot_log)

sx_from_com=zeros(3,length(com_trjx))
sy_from_com=zeros(3,length(com_trjy))   #重心からみた支持脚位置
sz_from_com=[-zc for i in 1:length(com_trjx)*3]
sz_from_com=reshape(sz_from_com,3,length(com_trjx))

sx_from_w=reshape(real_sx,3,vpoint_sum)
sy_from_w=reshape(real_sy,3,vpoint_sum)
vz_end=-0.5 #遊脚が接地する速度 -0.1

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum) 
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
        end
    else
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
            
        end
        if(leg_sign[i]!=0)
            #脚を挙げないようにコメントアウト
            sx_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sx_from_com,xv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sy_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sy_from_com,yv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sz_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator_z(0.01,-zc+0.01,-zc,vz_end,1+step,Tsup+step,dt)
        end
        step=step+Tsup
    
        for s in 1:Tdl
            sx_from_com[:,s+step]=sx_from_w[:,i]-com_trjx[s+step]
            sy_from_com[:,s+step]=sy_from_w[:,i]-com_trjy[s+step]
        end
        step=step+Tdl
    end
end
;
#-1が使われている可能性あり

In [11]:
function generate_csv(dataframe)
    CSV.write("leg_trj.csv",dataframe,delim=',',header=false)
end

generate_csv (generic function with 1 method)

In [12]:
df = DataFrame(xr=sx_from_com[1,:],
                xc=sx_from_com[2,:],
                xl=sx_from_com[3,:],
                yr=sy_from_com[1,:],
                yc=sy_from_com[2,:],
                yl=sy_from_com[3,:],
                zr=sz_from_com[1,:],
                zc=sz_from_com[2,:],
                zl=sz_from_com[3,:])
df=generate_csv(df)

CSV.Sink(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: Base.Dates.DateFormat(Base.Dates.Slot[Base.Dates.DelimitedSlot{Base.Dates.Year}(Base.Dates.Year,'y',4,"-"),Base.Dates.DelimitedSlot{Base.Dates.Month}(Base.Dates.Month,'m',2,"-"),Base.Dates.DelimitedSlot{Base.Dates.Day}(Base.Dates.Day,'d',2,r"(?=\s|$)")],"","english"),IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1),"leg_trj.csv",0,false,String["xr","xc","xl","yr","yc","yl","zr","zc","zl"],false)

In [13]:
a = []

0-element Array{Any,1}

###### 